In [8]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


## Set Parameters

NOTE: 1m data is only retrievable for the last 7 days, and any interval <1d is only for the last 60 days

In [11]:
startDate = "2010-01-01"
endDate = "2025-01-01"
symbol1 = "AAPL"
symbol2 = "MSFT"
timeInterval = "1d"

## Setup Data

In [13]:
# Download data for two stocks between specified dates with specified interval
data = yf.download([symbol1, symbol2], start=startDate, end=endDate, interval=timeInterval, auto_adjust=True)
# Get adjusted close prices
close = data["Close"]
close.head()

[*********************100%***********************]  2 of 2 completed


Ticker,AAPL,MSFT
Date,,
2010-01-04,6.431899,23.211437
2010-01-05,6.443016,23.218941
2010-01-06,6.340533,23.076445
2010-01-07,6.328811,22.836456
2010-01-08,6.370885,22.993952
